# **Notebook for Home Price Predictions** #


<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/ageron/handson-ml/blob/master/01_the_machine_learning_landscape.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## **Setup**

In [3]:
from google.colab import drive # import drive from google colab
 
ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)
 
drive.mount(ROOT)

/content/drive
Mounted at /content/drive


In [4]:
%cd drive/MyDrive/kaggle-home-prices/

/content/drive/MyDrive/kaggle-home-prices


In [ ]:
!pip install scikit-learn-intelex -q --progress-bar off
!pip install optuna

## **Load Data**

In [6]:
import numpy as np # linear algebra
import pandas as pd 
import os
 
os.chdir("data")

In [7]:
import sklearn
import seaborn as sns
from xgboost import XGBRegressor
# work around for installing sklearnrex
import sys
import os
import site
sys.path.append(os.path.join(os.path.dirname(site.getsitepackages()[0]), "site-packages"))
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [8]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

In [9]:
from sklearn.model_selection import train_test_split
practice_train_set, practice_test_set = train_test_split(train, test_size=0.2, random_state=42)

## **Transformers**

In [10]:
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn_pandas import CategoricalImputer
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor


class DataFrameSelector(BaseEstimator, TransformerMixin):
  def __init__(self, attribute_names):
    self.attribute_names = attribute_names
  def fit(self, X, y=None):
    return self  
  def transform(self, X):
    return X[self.attribute_names].values  


y = train["SalePrice"].copy()
X = train.drop("SalePrice", axis=1)

num_attribs = list(X.select_dtypes(exclude='object').keys())
cat_attribs = list(X.select_dtypes(include='object').keys())

y_practice = practice_train_set["SalePrice"].copy()
X_practice = practice_train_set.drop("SalePrice", axis=1)
y_practice_test = practice_test_set["SalePrice"].copy()
X_practice_test = practice_test_set.drop("SalePrice", axis=1)

num_pipeline= Pipeline([('selector', DataFrameSelector(num_attribs)), ('imputer', SimpleImputer(strategy='median')), ('std_scaler', StandardScaler()), ('feature_selector', SelectFromModel(RandomForestRegressor(), threshold=-np.inf, max_features=10)), ])
cat_pipeline= Pipeline([('selector', DataFrameSelector(cat_attribs)), ('imputer', CategoricalImputer()), ('one_hot_encoder', OneHotEncoder(sparse=False, handle_unknown='ignore')), ])
full_pipeline = FeatureUnion(transformer_list=[("num_pipeline", num_pipeline), ("cat_pipeline", cat_pipeline),])

X_train = full_pipeline.fit_transform(X_practice, y_practice)
X_test = full_pipeline.transform(X_practice_test)

In [13]:
X = full_pipeline.fit_transform(X, y)

In [14]:
test_X = full_pipeline.transform(test)
test_y = train["SalePrice"].copy()

##**Helper Functions**

###**Generate RMSE**

In [11]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

def get_RMSE(model, y, X):
  preds = model.predict(X)
  lin_mse = mean_squared_error(y, preds)
  lin_rmse = np.sqrt(lin_mse)
  print("RMSE", lin_rmse)
  lin_log_mse = mean_squared_error(np.log(y), np.log(preds))
  lin_log_rmse = np.sqrt(lin_log_mse)
  print("log RMSE", lin_log_rmse)

###**Write to CSV**

In [12]:
import csv

def write_to_csv(predictions, filename):
  # open the file in the write mode
  f = open(filename, 'w+')
  writer = csv.writer(f)
  writer.writerow(["Id", "SalePrice"])
  for count, id in enumerate(test["Id"]):
    writer.writerow([id, predictions[count]])
  f.close()  

##**SupportVectorRegressor**

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_log_error
import optuna
def objective_svr(trial):
    params ={
        'max_iter':trial.suggest_float('max_iter', 2000, 5000),
        'C':  trial.suggest_float('C', 1000, 3000),
    }
    model = LinearSVR(**params).fit(X_train, y_practice)
    y_pred = model.predict(X_test)
    loss = np.sqrt(mean_squared_log_error(y_practice_test, y_pred))
    return loss
svr_study = optuna.create_study(sampler=optuna.samplers.TPESampler(seed=123),
                            direction="minimize",
                            pruner=optuna.pruners.HyperbandPruner()) 

[I 2021-09-11 04:41:02,474] A new study created in memory with name: no-name-09f6fe8e-58d8-4b0e-86ec-cb1a113c3fe7


In [ ]:
svr_study.optimize(objective_svr, n_trials=1000)

In [22]:
import pickle
svr_model = LinearSVR(**svr_study.best_params).fit(X, y)
final_pred = svr_model.predict(test_X)
#write_to_csv(final_pred, "optuna_svr_submission.csv")
filename = 'optuna_svr_model.sav'
pickle.dump(svr_model, open(filename, 'wb'))

##**GradientBoostingRegressor**

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_log_error
import optuna
def objective_gbr(trial):
    params ={
        'n_estimators': trial.suggest_int('n_estimators', 140, 165),
        'max_features':  trial.suggest_int('max_features', 10, 12),
        'max_depth':  trial.suggest_int('max_depth', 4, 8),
        'learning_rate':  trial.suggest_float('learning_rate', 0.08, 0.3),
    }
    model = GradientBoostingRegressor(**params).fit(X_train, y_practice)
    y_pred = model.predict(X_test)
    loss = np.sqrt(mean_squared_log_error(y_practice_test, y_pred))
    return loss
gbr_study = optuna.create_study(sampler=optuna.samplers.TPESampler(seed=123),
                            direction="minimize",
                            pruner=optuna.pruners.HyperbandPruner())

[I 2021-09-10 05:51:06,340] A new study created in memory with name: no-name-17e7052f-5a98-462a-a91e-f4a40b6f12ee


In [ ]:
study.optimize(objective_gbr, n_trials=1000)

[I 2021-09-10 05:51:57,292] Trial 498 finished with value: 0.14514185205798202 and parameters: {'n_estimators': 159, 'max_features': 11, 'max_depth': 5, 'learning_rate': 0.18463995650397402}. Best is trial 326 with value: 0.13883308136251335.
[I 2021-09-10 05:51:57,489] Trial 499 finished with value: 0.145076782637612 and parameters: {'n_estimators': 159, 'max_features': 11, 'max_depth': 5, 'learning_rate': 0.18599293368803219}. Best is trial 326 with value: 0.13883308136251335.
[I 2021-09-10 05:51:57,685] Trial 500 finished with value: 0.1633574276639377 and parameters: {'n_estimators': 159, 'max_features': 11, 'max_depth': 5, 'learning_rate': 0.1826761200769223}. Best is trial 326 with value: 0.13883308136251335.
[I 2021-09-10 05:51:57,893] Trial 501 finished with value: 0.15877354700661248 and parameters: {'n_estimators': 160, 'max_features': 11, 'max_depth': 5, 'learning_rate': 0.1662936863627676}. Best is trial 326 with value: 0.13883308136251335.
[I 2021-09-10 05:51:58,093] Trial

In [28]:
import pickle
from sklearn.ensemble import GradientBoostingRegressor

gbr_model = GradientBoostingRegressor(n_estimators=160,max_features=11,max_depth=5,learning_rate=0.16514352203246263).fit(X, y)
final_pred = gbr_model.predict(test_X)
write_to_csv(final_pred, "optuna_gbr_submission.csv")
filename = 'optuna_gbr_model.sav'
pickle.dump(gbr_model, open(filename, 'wb'))

##**XGBRegressor**

In [21]:
 from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_log_error
import optuna
def objective_xgb(trial):
    params ={
        'gpu_id': 0,
        'tree_method': 'gpu_hist',
        'n_estimators': trial.suggest_int('n_estimators', 300, 400),
        'subsample': trial.suggest_float('subsample', 0.7, 0.9),
        'max_depth': 3,
        'learning_rate':  trial.suggest_float('learning_rate', 0.01, 0.1),
    }
    model = XGBRegressor(**params).fit(X_train, y_practice)
    y_pred = model.predict(X_test)
    loss = np.sqrt(mean_squared_log_error(y_practice_test, y_pred))
    return loss
xgb_study = optuna.create_study(sampler=optuna.samplers.TPESampler(seed=123),
                            direction="minimize",
                            pruner=optuna.pruners.HyperbandPruner())

[I 2021-09-11 04:01:54,974] A new study created in memory with name: no-name-60018684-d43d-4fda-9fd9-c493f9c948e8


In [22]:
 xgb_study.optimize(objective_xgb, n_trials=1000)

[04:01:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2021-09-11 04:01:59,115] Trial 0 finished with value: 0.1419529937604435 and parameters: {'n_estimators': 370, 'subsample': 0.7572278669900758, 'learning_rate': 0.030416630820778286}. Best is trial 0 with value: 0.1419529937604435.


[04:01:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2021-09-11 04:02:01,226] Trial 1 finished with value: 0.14055690353613923 and parameters: {'n_estimators': 355, 'subsample': 0.8438937939571126, 'learning_rate': 0.04807958141120149}. Best is trial 1 with value: 0.14055690353613923.


[04:02:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

[I 2021-09-11 04:22:59,145] Trial 572 finished with value: 0.13523896252084017 and parameters: {'n_estimators': 373, 'subsample': 0.7816796711636547, 'learning_rate': 0.054288459360046976}. Best is trial 215 with value: 0.13094630252079567.


[04:22:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2021-09-11 04:23:01,180] Trial 573 finished with value: 0.13783230484766532 and parameters: {'n_estimators': 343, 'subsample': 0.770396922506207, 'learning_rate': 0.05866327574050016}. Best is trial 215 with value: 0.13094630252079567.


[04:23:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


KeyboardInterrupt: ignored

In [30]:
import pickle
xgb_model = XGBRegressor(**xgb_study.best_params).fit(X, y)
final_pred_xgb = gbr_model.predict(test_X)
write_to_csv(final_pred_xgb, "optuna_xgb_submission.csv")
filename = 'optuna_xgb_model.sav'
pickle.dump(xgb_model, open(filename, 'wb'))

[04:25:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


##**StackedModel**##

In [24]:
import joblib
xgb = joblib.load('optuna_xgb_model.sav')
gbr = joblib.load('optuna_gbr_model.sav')
svr = joblib.load('optuna_svr_model.sav')

[04:51:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [31]:
from mlxtend.regressor import StackingCVRegressor

stack_gen = StackingCVRegressor(regressors=(xgb, gbr, svr),
                                meta_regressor=xgb,
                                use_features_in_secondary=True)

In [ ]:
xgb_final_preds = xgb.predict(test_X)
gbr_final_preds = gbr.predict(test_X)
svr_final_preds = svr.predict(test_X)
stack_gen_final_preds = stack_gen.predict(test_X)

In [ ]:
y3 = exp(  (log(xgb_final_preds) + log(gbr_final_preds) + log(svr_final_preds) + log(stack_gen_final_preds)) / 4 )